In [3]:
import time
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
import numpy as np
import json
import pandas as pd
import autokeras as ak

from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from typing_extensions import override

"""
The datasets folder demands that this structures is followed:
datasets/
   <FILE_NAME>.csv # where the main tabular data is stored
   /<ALGORITHM_NAME_1>/ # where the algorithm specific data is stored
       key.csv # where the key for the images are stored
       <IMAGE_NAME>.json # the images are stored as an array
   /<ALGORITHM_NAME_2>/ # where the algorithm specific data is stored
       ...

The pipeline will then run the <FILE_NAME>.csv file through a number of traditional models
and algorithms. While the converted images are run through a autokeras CNN pipeline.

All the results are then saved in the
results/<DATASET_NAME>/<EXPERIMENT_NAME>...
The following are stored
- results.csv # contains the metrics for all the algorithms tested (final results)
- <ALGORITHM_NAME>.pkl # contains the trained model for the algorithm
"""
DATASETS = [
    {
        "path": "datasets/sample",
        "filename": "Simple_Classification_Dataset.csv",
        "target_column": "label"
    }
]

TRADITIONAL_MODELS = {
    "RF": {
        "class": RandomForestClassifier,
        "params": {
            "n_estimators": 100,
            "verbose": 0,
        }
    },
    "SVM": {
        "class": SVC,
        "params": {
            "kernel": "rbf",
            "C": 1.0,
            "gamma": 'scale',
            "verbose": 0,
        }
    },
}

In [4]:
def dataloader(path):
    """
    Load the images based on the key.csv file.
    Creating a numpy array of images and a list of labels.
    :return: (X, y) where X is numpy array of shape (num_samples, height, width, channels)
    """
    # load the key.csv file
    df = pd.read_csv(os.path.join(path, "key.csv"))

    x = []
    y = []

    for index, row in df.iterrows():
        filename = row['filename'].replace('.png', '.json')
        image_path = os.path.join(path, filename)

        # Load the JSON data
        with open(image_path, 'r') as f:
            image_data = json.load(f)

        # Convert to proper numpy array
        if isinstance(image_data, list):
            # If it's a flat list of RGB values, reshape appropriately
            image_array = np.array(image_data)

            # Reshape to proper image dimensions (you'll need to know your image dimensions)
            # For example, if it's a 28x28 RGB image:
            # image_array = image_array.reshape(28, 28, 3)

        else:
            # If it's already structured, convert directly
            image_array = np.array(image_data)

        # Ensure proper data type and range
        if image_array.max() > 1.0:
            image_array = image_array.astype(np.float32) / 255.0  # Normalize to [0,1]

        x.append(image_array)
        y.append(row['label'])

    # Convert to numpy arrays
    X = np.array(x)  # Shape should be (num_samples, height, width, channels)
    y = np.array(y)  # Shape should be (num_samples,)

    return X, y


def autokeras_runner(path, n=1, epochs=10, batch_size=None, validation_split=None,
                     validation_data=None, max_trials=1, directory='auto_keras',
                     algorithm_name='image_classifier', overwrite=True, seed=None,
                     max_model_size=None, tuner='random', **kwargs):
    """
    Load the dataset from the given path and run AutoKeras with configurable parameters.
    """
    # get the data
    x, y = dataloader(path)

    results_list = []

    # Set default batch_size if not provided
    if batch_size is None:
        batch_size = min(32, len(x) // 2) if len(x) < 64 else 32

    # Set default validation if not provided
    if validation_split is None and validation_data is None:
        if len(x) < 64:
            validation_data = (x, y)
        else:
            validation_split = 0.2

    for run in range(n):
        start_time = time.time()

        clf = ak.ImageClassifier(
            max_trials=max_trials,
            directory=directory,
            project_name=f"{algorithm_name}_run_{run}",
            overwrite=overwrite,
            seed=seed,
            max_model_size=max_model_size,
            tuner=tuner
        )

        fit_kwargs = {
            'epochs': epochs,
            'batch_size': batch_size,
            **kwargs
        }

        if validation_data is not None:
            fit_kwargs['validation_data'] = validation_data
        elif validation_split is not None:
            fit_kwargs['validation_split'] = validation_split

        clf.fit(x, y, **fit_kwargs)
        train_time = time.time() - start_time

        model = clf.export_model()
        y_pred = model.predict(x)
        y_pred = np.argmax(y_pred, axis=1)

        # calculate metrics
        accuracy = accuracy_score(y, y_pred)
        precision = precision_score(y, y_pred, average='weighted')
        recall = recall_score(y, y_pred, average='weighted')
        f1 = f1_score(y, y_pred, average='weighted')

        results_list.append({
            'model_name': 'autokeras_cnn',
            'run': run + 1,
            'accuracy': accuracy,
            'precision': precision,
            'recall': recall,
            'f1_score': f1,
            'train_time': train_time
        })

    return results_list


def get_algorithm_dir(path):
    algorithm_dirs = [d for d in os.listdir(dataset["path"]) if os.path.isdir(os.path.join(dataset["path"], d))]
    # for each of those algorithm dir check if there is a key.csv file if not remove it from the list
    algorithm_dirs = [d for d in algorithm_dirs if os.path.isfile(os.path.join(path, d, "key.csv"))]

    return algorithm_dirs


def autokeras_cnn_pipeline(path, output_folder, all_result_df, all_result_df_path=None):
    """
    Run the autokeras CNN pipeline on the dataset.
    The dataset should be a CSV file with a target column.
    """
    # get the list of all algorithm in the dataset folder
    algorithms = get_algorithm_dir(path)

    # for each algorithm run the autokeras CNN pipeline
    for algorithm in algorithms:
        # get the path to the algorithm folder
        algorithm_path = os.path.join(path, algorithm)

        # check if the key.csv file exists
        result = autokeras_runner(algorithm_path, directory=output_folder, algorithm_name=algorithm)
        # convert the results to a DataFrame
        result_df = pd.DataFrame(result)
        # merge the results with the main DataFrame
        all_result_df = pd.concat([all_result_df, result_df], ignore_index=True)
        # save the result df to a CSV file at interval to make sure we don't lose results
        all_result_df.to_csv(all_result_df_path, index=False, overwrite=True)

    # convert the results to a DataFrame
    return all_result_df.copy()

In [5]:
from libs.traditional_classification_pipeline import tabular_testing
import os
from datetime import datetime

for dataset in DATASETS:
    # for a given dataset create a new experiment folder
    output_folder = f"results/{dataset['path'].split('/')[-1]}/{datetime.now().strftime('%Y%m%d-%H%M%S')}"
    os.makedirs(output_folder, exist_ok=True)

    # try to run the traditional models first
    results = tabular_testing(file_name=os.path.join(dataset["path"], dataset["filename"]), target_column=dataset["target_column"], model_configs=TRADITIONAL_MODELS)
    results.to_csv(f"{output_folder}/results.csv", index=False, overwrite=True)
    # get all the "directories" in the dataset folder
    results = autokeras_cnn_pipeline(dataset["path"], output_folder=output_folder, all_result_df=results, all_result_df_path=f"{output_folder}/results.csv")

    # save the results
    #results = tab_result
    results.to_csv(f"{output_folder}/results.csv", index=False, overwrite=True)
print("DONE!!!")

Trial 1 Complete [00h 00m 42s]
val_loss: 0.6931465268135071

Best val_loss So Far: 0.6931465268135071
Total elapsed time: 00h 00m 42s
Epoch 1/10
2/2 ━━━━━━━━━━━━━━━━━━━━ 7s 1s/step - accuracy: 0.2667 - loss: 0.7487 - val_accuracy: 0.5000 - val_loss: 0.6931
Epoch 2/10
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 260ms/step - accuracy: 0.4667 - loss: 0.6968 - val_accuracy: 0.5000 - val_loss: 0.6931
Epoch 3/10
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 267ms/step - accuracy: 0.6000 - loss: 0.6693 - val_accuracy: 0.5000 - val_loss: 0.6931
Epoch 4/10
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 278ms/step - accuracy: 0.9333 - loss: 0.5662 - val_accuracy: 0.5000 - val_loss: 0.6931
Epoch 5/10
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 280ms/step - accuracy: 0.5333 - loss: 0.6793 - val_accuracy: 0.5000 - val_loss: 0.6931
Epoch 6/10
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 258ms/step - accuracy: 0.6000 - loss: 0.6794 - val_accuracy: 0.5000 - val_loss: 0.6931
Epoch 7/10
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 270ms/step - accuracy: 0.6000 - loss: 0.6234 - val_accuracy: 0.5000 - val_

D:\Python Project\Tab2Img\.venv\Lib\site-packages\sklearn\metrics\_classification.py:1706: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape[0])



Search: Running Trial #1

Value             |Best Value So Far |Hyperparameter
True              |True              |image_block_1/normalize
False             |False             |image_block_1/augment
xception          |xception          |image_block_1/block_type
global_avg        |global_avg        |classification_head_1/spatial_reduction_1/reduction_type
0.25              |0.25              |classification_head_1/dropout
sgd               |sgd               |optimizer
0.0001            |0.0001            |learning_rate

Epoch 1/10
2/2 ━━━━━━━━━━━━━━━━━━━━ 7s 1s/step - accuracy: 0.6000 - loss: 0.6718 - val_accuracy: 0.6000 - val_loss: 0.6931
Epoch 2/10
2/2 ━━━━━━━━━━━━━━━━━━━━ 7s 6s/step - accuracy: 0.4667 - loss: 0.6761 - val_accuracy: 0.5000 - val_loss: 0.6931
Epoch 3/10
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 263ms/step - accuracy: 0.6000 - loss: 0.6527 - val_accuracy: 0.5000 - val_loss: 0.6931
Epoch 4/10
2/2 ━━━━━━━━━━━━━━━━━━━━ 4s 4s/step - accuracy: 0.6000 - loss: 0.6465 - val_accuracy: 0.


KeyboardInterrupt

